In [ ]:
import random
from socket import *
import threading

# Set up the socket

serversocket = socket(AF_INET, SOCK_STREAM)
serversocket.bind(('localhost', 4000))
serversocket.listen(5)

# Function definitions

def handleclient(clientsocket, clientaddress):
    # Receive the client's greeting
    clientgreeting = clientsocket.recv(1024)
    # Send the welcome message to the client
    welcomemessage = "Greetings\r\n"
    clientsocket.send(welcomemessage.encode('ascii'))

    running = 1
    guesses = []
    # Generate a random number for the client to try and guess
    answer = generatenumber()
    # Main loop
    while running:
        guess = clientsocket.recv(1024)

        guessstring = guess.decode('ascii')
        print(guessstring)
        # Split the guess string up to get the integer guessed
        guess = int(guessstring.split()[1])

        running = True
        
        diff = guess - answer
        guesses.append(guess)
        good_guesses = [g for g in guesses if g <= 100 and g >= 1]
        
        # If the player has guessed correctly
        if (guess == answer):
            messagetosend = (f'You got it! It only took you {len(guesses)} attempts...')
            clientsocket.send(messagetosend.encode('ascii'))
            running = False
        else:
            if guess > 100 or guess < 1:
                messagetosend = ('OUT OF BOUNDS\r\n')

            if (guess > 100 or guess < 1) and len(good_guesses) > 1:
                messagetosend = (f'Your last good guess was {good_guesses[-1]}\r\n')

            elif len(good_guesses) == 1:
                if abs(diff) <= 10:
                    messagetosend = ('WARM!\r\n')
                else:
                    messagetosend = ('COLD!\r\n')

            elif len(good_guesses) > 1:
                if abs(answer - good_guesses[-1]) < abs(answer - good_guesses[-2]):
                    messagetosend = ('WARMER...\r\n')
       
                else:
                    messagetosend = ('COLDER...\r\n')
                
            # Send the response to the player
        clientsocket.send(messagetosend.encode('ascii'))
    # Close the connection
    clientsocket.close()
    print("Connection closed.")

def generatenumber():
    return random.randrange(1, 100)

# Main server loop

while True:
    (clientsocket, clientaddress) = serversocket.accept()
    print("Connection received from: ", clientaddress)
    threading.Thread(target = handleclient, args =  (clientsocket, clientaddress)).start()
    print("Connection passed to new thread. Returning to listening.")

Connection received from:  ('127.0.0.1', 62462)
Connection passed to new thread. Returning to listening.
